In [1]:
import tensorflow as tf
import numpy as np
import gym
import gym.spaces

/home/martin/anaconda3/envs/pysc2/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
env = gym.make("LunarLander-v2")

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


In [3]:
class Agent:
    learning_rate = 0.001
    def __init__(self, name, input_layer, hsize, action_space_n):
        self.name = name
        self.hsize = hsize
        self.action_space_n = action_space_n
        with tf.variable_scope(name,reuse=False):
            self.input = input_layer
            self.h1 = tf.layers.dense(input_layer,hsize[0],activation=tf.nn.leaky_relu)
            self.h2 = tf.layers.dense(self.h1,hsize[1],activation=tf.nn.leaky_relu)
            self.out = tf.layers.dense(self.h2, action_space_n, activation=tf.nn.softmax)
            
    def zero(self):
        with tf.variable_scope(self.name,reuse=False):
            self.h1 = tf.zeros_like(self.h1)
            self.h2 = tf.zeros_like(self.h2)
            self.out = tf.zeros_like(self.out)
        
    def from_population(self, agents, weights):
        self.zero()
        weight = 1. / len(agents)
        train_vars = self.train_vars()
        # for i in range(len(agents)):
        for i in range(1):
            current_agent_vars = agents[i].train_vars()
            # for j in range(len(current_agent_vars)):
            for j in range(1):
                print("train_vars[j]:\n", sess.run(train_vars[j]))
                print("weights[i]:\n", weights[i])
                print("current_agent_vars[j]:\n", sess.run(current_agent_vars[j]))
                print("train_vars[j] + weights[i] * current_agent_vars[j]:\n", sess.run(train_vars[j] + weights[i] * current_agent_vars[j]))
                tf.assign(train_vars[j], train_vars[j] + weights[i] * current_agent_vars[j])
                print("train_vars[j]:\n", sess.run(train_vars[j]))
                
        
    def compute(self, observation):
        obs_np = np.array(observation).reshape((1, 8))
        return sess.run(self.out, feed_dict= {X : obs_np})
    
    def train_vars(self):
        return tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, self.name)
    
    def update(self, agent):
        pass

In [4]:
def agent(name, input_layer,hsize, reuse, action_space_n):
    with tf.variable_scope(name,reuse=False):
        h1 = tf.layers.dense(input_layer,hsize[0],activation=tf.nn.leaky_relu)
        h2 = tf.layers.dense(h1,hsize[1],activation=tf.nn.leaky_relu)
        out = tf.layers.dense(h2, action_space_n, activation=tf.nn.softmax)

    return name, h1, h2, out

In [5]:
num_agents = 2
num_games_per_agent = 100
max_steps = 10
hsizes = [24, 24]
alpha = 0.03

In [6]:
X = tf.placeholder(tf.float32,[None, env.observation_space.shape[0]])

In [7]:
smith = Agent(name="Agent_Smith", input_layer=X, hsize=hsizes, action_space_n=env.action_space.n)
print(smith.train_vars())
agents = []
for i in range(num_agents):
    # agents.append(agent("agent/%d" % (i,), input_layer=X, hsize=hsizes, reuse=False, action_space_n=env.action_space.n))
    agents.append(Agent("agent/%d" % (i,), input_layer=X, hsize=hsizes, action_space_n=env.action_space.n))

[<tf.Variable 'Agent_Smith/dense/kernel:0' shape=(8, 24) dtype=float32_ref>, <tf.Variable 'Agent_Smith/dense/bias:0' shape=(24,) dtype=float32_ref>, <tf.Variable 'Agent_Smith/dense_1/kernel:0' shape=(24, 24) dtype=float32_ref>, <tf.Variable 'Agent_Smith/dense_1/bias:0' shape=(24,) dtype=float32_ref>, <tf.Variable 'Agent_Smith/dense_2/kernel:0' shape=(24, 4) dtype=float32_ref>, <tf.Variable 'Agent_Smith/dense_2/bias:0' shape=(4,) dtype=float32_ref>]


In [8]:
sess = tf.Session()
tf.global_variables_initializer().run(session=sess)

In [9]:
#for i in range(num_agents):
#    train_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, "agent/%d" % (i,))
#    print(sess.run(train_vars))
#    print("======================")

In [10]:
def play(env, agent, num_games, max_steps, render):
    rewards = np.zeros([num_games])
    actions = [_ for _ in range(env.action_space.n)]
    for g in range(num_games):
        obs = env.reset()
        #print(obs)
        reward_accum = 0
        for i in range(max_steps):
            # obs_np = np.array(obs).reshape((1, 8))
            # action_probabilities = sess.run(agent, feed_dict= {X : obs_np})
            action_probabilities = agent.compute(obs)[0]
            
            #obs, r, done, info = env.step(env.action_space.sample())
            action = np.random.choice(a=actions, p=action_probabilities)
            obs, r, done, info = env.step(action)
            if render == True:
                env.render()
            reward_accum += r
        rewards[g] = reward_accum
    return rewards.sum()

In [11]:
def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum()

In [12]:
rewards_np = np.zeros([num_agents])
for i in range(num_agents):
    rewards_np[i] = play(env=env, agent=agents[i], num_games=num_games_per_agent, max_steps=max_steps, render=False)
print(rewards_np)
weights = np.ones((num_agents, )) / num_agents
print("weights=", weights)
smith.from_population(agents=agents, weights = np.ones((num_agents, )) / num_agents  )

[-638.51898089 -688.0985127 ]
weights= [0.5 0.5]
train_vars[j]:
 [[ 0.0334681   0.031775    0.38692448 -0.17300358  0.21356097 -0.3047756
  -0.06983069  0.03076768 -0.07317385  0.30405304  0.109409   -0.33886042
  -0.25564736  0.34990814  0.07520857 -0.19894046  0.2556351   0.0844427
  -0.34792352 -0.04074609  0.32809517 -0.27815664 -0.00625387  0.2789239 ]
 [ 0.2190552  -0.3164183   0.09314355 -0.11156267  0.31523517  0.35178855
  -0.24131471  0.3840274   0.16065606 -0.00451037 -0.38510013  0.34006694
   0.3544356  -0.3950615   0.15713641  0.19371018  0.38668802 -0.27587104
   0.41631463  0.03639954  0.15049139 -0.00449324  0.18760076  0.28374162]
 [ 0.07043216 -0.2888958  -0.05788901  0.1964328  -0.02508587  0.37817088
  -0.24578308  0.02347711  0.09816918  0.28136125  0.189661    0.11520287
  -0.24966803  0.4304435   0.25214598  0.39891896  0.11995474  0.03839564
   0.24294218  0.13491663  0.42220405 -0.37226388 -0.12359458  0.2030159 ]
 [-0.19877476 -0.13802904 -0.33577204 -0.21449

In [13]:
print(sess.run(agents[0].train_vars()[0]))
print("====================")
print(sess.run(agents[1].train_vars()[0]))
print("====================")
print(sess.run(smith.train_vars()[0]))
print("====================")
print(sess.run(0.5 * agents[0].train_vars()[0] + 0.5 * agents[1].train_vars()[0]))


[[-0.3701759  -0.24014626  0.40834978  0.15947422 -0.07912192  0.2488598
  -0.03451741 -0.11296847 -0.08187383 -0.0671306   0.12547019  0.38220325
   0.2121239   0.03027463 -0.0899325   0.37464312  0.3347182  -0.28120092
   0.02518633  0.14185765  0.10088482 -0.39950776 -0.262693   -0.3144077 ]
 [-0.3655529   0.4010276   0.33596286  0.07631758 -0.13041338 -0.22017317
   0.12361947  0.32485315 -0.02124262  0.00444597 -0.1763286  -0.35869053
   0.38969144 -0.3200912  -0.29208845 -0.33015466 -0.2293975  -0.4313132
   0.13255545 -0.3137795  -0.3451853  -0.283257   -0.0212934   0.18290743]
 [-0.05560404  0.22949705  0.2212623   0.05347061 -0.13132238 -0.33896366
  -0.27927542  0.15537414 -0.12726462  0.0724065   0.00617912  0.06927022
   0.23001418  0.34492245 -0.38892013 -0.07372668  0.07473442  0.06140095
   0.11153606 -0.39178014 -0.2870601  -0.30872333 -0.04817212 -0.30241358]
 [-0.35381097  0.13585714  0.14617828  0.2354103  -0.23505114 -0.40136686
   0.23027852  0.07247511  0.250496  

In [14]:
print (sorted(rewards_np))

[-688.098512699813, -638.5189808884707]


In [15]:
o = env.reset()
for i in range(200):
    o, r, done, info = env.step(env.action_space.sample())
    env.render()
    print(o, r, done, info)
    #if done:
    #    break

[-0.01452866  0.94882365 -0.73479238  0.26077909  0.0166598   0.16473056
  0.          0.        ] -0.23348306038960231 False {}
[-0.02170115  0.95234299 -0.72320619  0.23453754  0.02255979  0.11801085
  0.          0.        ] 0.9554331647779872 False {}
[-0.02893887  0.95545741 -0.73141499  0.20749434  0.03010574  0.15093311
  0.          0.        ] -1.1142568077491137 False {}
[-0.03617706  0.95817233 -0.73143859  0.18082212  0.03764925  0.15088443
  0.          0.        ] -0.36836144945024785 False {}
[-0.04334297  0.96048794 -0.72238078  0.15421298  0.04337157  0.11445717
  0.          0.        ] 0.6167383734091356 False {}
[-0.05044823  0.9623992  -0.71475658  0.12729231  0.04756765  0.08392902
  0.          0.        ] 0.5903827680565723 False {}
[-0.05748034  0.96391736 -0.70558419  0.10113446  0.0499146   0.04694334
  0.          0.        ] 0.8651504113233284 False {}
[-0.06444969  0.96504021 -0.69770656  0.0748309   0.05067628  0.01523518
  0.          0.        ] 0.84654

In [16]:
env.action_space.n

4

In [17]:
env.action_space.sample()

1